In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD

2024-01-14 20:10:48.041770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-14 20:10:48.510671: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-14 20:10:48.511606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-14 20:10:50.406822: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('~/MITM-Detection/dataset.csv')

In [3]:

# Display the first few rows of the DataFrame
print(df.head())

# Extract features and labels
X = df.iloc[:, :-1]  # Features (all columns except the last one)
y = df.iloc[:, -1]   # Labels (last column)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape dữ liệu để phù hợp với đầu vào của Conv1D
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

           0  6.666666667  0.1  0.2  0.3
0   8.000000    61.666667   24    0    0
1  18.000000    43.333333   54    0    0
2  23.666667    41.333333   71    0    0
3  37.666667    38.333333   91    0    0
4  52.333333    25.000000   95    0    0


In [4]:
# model = Sequential()
# model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))

In [5]:
# # create and fit the LSTM network
# model = Sequential()
# model.add(LSTM(4, input_shape=(X_train.shape[1], 1)))
# model.add(Dense(1))
# model.compile(loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=2, validation_data=(X_test, y_test))

# # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # # Huấn luyện mô hình
# # model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# # Dự đoán trên tập kiểm tra
# y_pred = model.predict(X_test)
# y_pred_binary = (y_pred > 0.5).astype(int)

In [6]:
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Second GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Third GRU layer
regressorGRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Fourth GRU layer
regressorGRU.add(GRU(units=50, activation='tanh'))
regressorGRU.add(Dropout(0.2))
# The output layer
regressorGRU.add(Dense(units=1))
# Compiling the RNN
regressorGRU.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
# Fitting to the training set
regressorGRU.fit(X_train,y_train,epochs=100,batch_size=32, validation_data=(X_test, y_test))
y_pred = regressorGRU.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

Epoch 1/100
1179/1179 [==============================] - 36s 20ms/step - loss: 0.3935 - accuracy: 0.8989 - val_loss: 0.3314 - val_accuracy: 0.9699
Epoch 2/100
1179/1179 [==============================] - 21s 18ms/step - loss: 0.2940 - accuracy: 0.9600 - val_loss: 0.2832 - val_accuracy: 0.9699
Epoch 3/100
1179/1179 [==============================] - 21s 18ms/step - loss: 0.2504 - accuracy: 0.9605 - val_loss: 0.2387 - val_accuracy: 0.9699
Epoch 4/100
1179/1179 [==============================] - 21s 18ms/step - loss: 0.2330 - accuracy: 0.9615 - val_loss: 0.1597 - val_accuracy: 0.9699
Epoch 5/100
1179/1179 [==============================] - 22s 18ms/step - loss: 0.2302 - accuracy: 0.9640 - val_loss: 0.1989 - val_accuracy: 0.9702
Epoch 6/100
1179/1179 [==============================] - 22s 19ms/step - loss: 0.2187 - accuracy: 0.9715 - val_loss: 0.2332 - val_accuracy: 0.9708
Epoch 7/100
1179/1179 [==============================] - 24s 20ms/step - loss: 0.2275 - accuracy: 0.9695 - val_loss: 0

In [7]:
# Tính confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_binary)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[6256    0]
 [ 239 2931]]


In [8]:
#Get the evaluation metrics like precision, recall, f1-score
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_binary))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6256
           1       1.00      0.92      0.96      3170

    accuracy                           0.97      9426
   macro avg       0.98      0.96      0.97      9426
weighted avg       0.98      0.97      0.97      9426

